In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
#import missingno as msno

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
import random
import warnings
warnings.filterwarnings('ignore')

# Test dosyası yüklenmesi

In [ ]:
import os
os.chdir(r"C:\Users\Asus\Desktop\Enerjisa_Bootcamp_Bitirme_Projesi_Grup1")

In [ ]:
df = pd.read_excel("train_5 - Kopya.xlsx")
df.head()

# Test dosyasının tahminlemeye hazılanması

In [ ]:
# eksik kategorik degerler doldurulur

df["SAYAC_MODEL"].fillna("UNDEFINED", inplace=True)
df["SAYAC_MARKA"].fillna("UNDEFINED", inplace=True)
df["SAYAC_OLCUM_TURU"].fillna("UNDEFINED", inplace=True)
df["SAYAC_FAZ_N"].fillna("UNDEFINED", inplace=True)
df["SAYAC_YAPIM_YILI"].fillna(df["SAYAC_YAPIM_YILI"].mean(), inplace=True)

In [ ]:
# kategorik degerler gruplanır

def sayac_marka(i):
    if i in ('UNDEFINED'):
        return 'Çok Yüksek Risk'
    elif i in ('LUNA'):
        return 'Yüksek Risk'
    else:
        return "Orta - Düşük Risk"
df['SAYAC_MARKA'] = df['SAYAC_MARKA'].apply(sayac_marka)


def sayac_yapim(i):
    if i >= 2015:
        return "2015 ve sonrası"
    else:
        return '2015 Öncesi'
df['SAYAC_YAPIM_YILI'] = df['SAYAC_YAPIM_YILI'].apply(sayac_yapim)

bins = [df["MAHALLE_RISK_SKORU"].min(), df["MAHALLE_RISK_SKORU"].median(), df["MAHALLE_RISK_SKORU"].quantile(0.75), df["MAHALLE_RISK_SKORU"].max()]
bin_labels = [ "Düşük", 'Orta', 'Yüksek']
df['MAHALLE_RISK_SKORU_group'] = pd.cut(df['MAHALLE_RISK_SKORU'], bins = bins, labels=bin_labels, include_lowest= True)

bins = [df["SOKAK_RISK_SKORU"].min(), df["SOKAK_RISK_SKORU"].quantile(0.75), df["SOKAK_RISK_SKORU"].max()]
bin_labels = ["Düşük", 'Yüksek']
df['SOKAK_RISK_SKORU_group'] = pd.cut(df['SOKAK_RISK_SKORU'], bins = bins, labels=bin_labels, include_lowest= True)

bins = [df["SOB_RISK_SKORU"].min(), df["SOB_RISK_SKORU"].quantile(0.25), df["SOB_RISK_SKORU"].median(), df["SOB_RISK_SKORU"].quantile(0.75), df["SOB_RISK_SKORU"].max()]
bin_labels = ["Çok Düşük", "Düşük", 'Orta', 'Yüksek']
df['SOB_RISK_SKORU_group'] = pd.cut(df['SOB_RISK_SKORU'], bins = bins, labels=bin_labels, include_lowest= True)

In [ ]:
# tuketim verileri üzerine işlemler yapılır

tuketim = (df.iloc[:,0:24])# tuketim degerlerini incelemek istersek

tuketim["aylik_ort_tuketim"] = np.nan
tuketim["max_tuketim"] = np.nan
tuketim["min_tuketim"] = np.nan
tuketim["max_aylik_tuketim_dususu"] = np.nan
tuketim["max_yillik_tuketim_dususu"] = np.nan
tuketim["tuketim_null_sayisi"] = np.nan

tuketim.fillna(-1, inplace=True) # veri setindeki tum null degerler gecici olarak -1 ile doldurulur
 

tuketim1 = tuketim.iloc[:,:24]
tuketim1

monthly_avg = 0.0
monthly_min = 999.00
monthly_max = 0.0

for i in range(len(tuketim1)): # satirlar arasi dolasir
    for month in tuketim1: # sutunlar arasi dolasir
            
        if(tuketim[month].values[i] != -1): # nulları -1 ile doldurmustuk onlari ort hesabina katmiyoruz
            monthly_avg += tuketim[month].values[i]
            
        if(tuketim[month].values[i] > monthly_max): # kullanicinin aylık max tuketimi hesaplanir
            monthly_max = tuketim[month].values[i]
            
        if(tuketim[month].values[i] < monthly_min and tuketim[month].values[i] != -1): # kullanicinin aylık min tuketimi hesaplanir
            monthly_min = tuketim[month].values[i]
            
        
    monthly_avg = monthly_avg/len(tuketim1.columns)
    
    tuketim["aylik_ort_tuketim"].values[i] = monthly_avg
    tuketim["max_tuketim"].values[i] = monthly_max
    tuketim["min_tuketim"].values[i] = monthly_min
    
    monthly_avg = 0.0
    monthly_min = 999.00
    monthly_max = 0.0
    temp = 0
    
counter = 0

for i in range(len(tuketim1)): # satirlar arasi dolasir
    for month in tuketim1: # sutunlar arasi dolasir  
        if(tuketim[month].values[i] == -1): # nulları -1 ile doldurmustuk, deger null ise buraya girer
            tuketim[month].values[i] = tuketim["aylik_ort_tuketim"].values[i] # -1 olan deger, ort ile degistirildi
            counter += 1
    tuketim["tuketim_null_sayisi"].values[i] = counter   
            
    counter = 0
    
consumption_diff = 0 
temp = 0

for i in range(len(tuketim1)): # satirlar arasi dolasir
    for month in tuketim1: # sutunlar arasi dolasir      
        if(temp < tuketim[month].values[i]): # düsüs varsa fark hesaplanir
            if(tuketim[month].values[i] != tuketim["aylik_ort_tuketim"].values[i] and temp != tuketim["aylik_ort_tuketim"].values[i] ): # degerler null degilse fark hesaplanir
                diff = abs(tuketim[month].values[i] - temp)    
                if(diff > consumption_diff and temp != 0):
                    consumption_diff = diff
            
        temp = tuketim[month].values[i]
        
    tuketim["max_aylik_tuketim_dususu"].values[i] = consumption_diff
    consumption_diff = 0 
    temp = 0
    
#tuketim icin aynı ayi yil bazinda karsilastirdim, farkini alinip yillik tuketim dususu hesaplanir

months = []
for x in tuketim1:
    months.append(x)

len_m = int(len(months)/2)

yearly_max = 0

for i in range(len(tuketim1)): # satir bazinda dolasir
    for j in range(len_m): # sutun bazinda dolasir
        month = months[j] # yakin tarih
        nmonth = months[j+12]
        if(tuketim[month].values[i] < tuketim[nmonth].values[i] ): # düsüs varsa fark hesapla
            if(tuketim[month].values[i] != tuketim["aylik_ort_tuketim"].values[i] and tuketim[nmonth].values[i] != tuketim["aylik_ort_tuketim"].values[i] ): # aylardan biri null degilse degisim hesapla
                diff = abs(tuketim[month].values[i] - tuketim[nmonth].values[i])
                if(diff > yearly_max):
                    yearly_max = diff
    tuketim["max_yillik_tuketim_dususu"].values[i] = yearly_max
    yearly_max = 0
    


In [ ]:
# demand verileri üzerine işlemler yapılır

demand = df.iloc[:,24:36].copy() # demand degerlerini incelemek istersek

# demande 5 yeni kolon ekle

demand["aylik_ort_demand"] = np.nan
demand["max_demand"] = np.nan
demand["min_demand"] = np.nan
demand["max_aylik_demand_degisimi"] = np.nan
demand["demand_null_sayisi"] = np.nan

demand.fillna(-1, inplace=True)

demand1 = demand.iloc[:,:12]

monthly_avg = 0.0
monthly_min = 999.00
monthly_max = 0.0
demand_diff = 0 
temp = 0


for i in range(len(demand1)): # satirlar arasi dolasir
    for month in demand1: # sutunlar arasi dolasir
        
        if(demand[month].values[i] != -1 and temp != -1): # degerler null degilse fark hesaplanir
            diff = abs(demand[month].values[i] - temp)
            if(diff > demand_diff and temp != 0):
                demand_diff = diff
            temp = demand[month].values[i]
            
        if(demand[month].values[i] != -1): # nulları -1 ile doldurmustuk onlari ort hesabina katmiyoruz
            monthly_avg += demand[month].values[i]
            
        if(demand[month].values[i] > monthly_max): # kullanicinin aylık max tuketimi hesaplanir
            monthly_max = demand[month].values[i]
            
        if(demand[month].values[i] < monthly_min and demand[month].values[i] != -1): # kullanicinin aylık min tuketimi hesaplanir
            monthly_min = demand[month].values[i]
            
        
    monthly_avg = monthly_avg/len(demand1.columns)
    
    demand["aylik_ort_demand"].values[i] = monthly_avg
    demand["max_demand"].values[i] = monthly_max
    demand["min_demand"].values[i] = monthly_min
    demand["max_aylik_demand_degisimi"].values[i] = demand_diff
    
    monthly_avg = 0.0
    monthly_min = 999.00
    monthly_max = 0.0
    demand_diff = 0 
    temp = 0.0
    

# null (-1) demand degerleri kullanicinin aylik ort demandi ile doldurulur, null sayisi hesaplanir

for i in range(len(demand1)): # satirlar arasi dolasir
    for month in demand1: # sutunlar arasi dolasir  
        if(demand[month].values[i] == -1): # nulları 0 ile doldurmustuk
            demand[month].values[i] = demand["aylik_ort_demand"].values[i] # -1 olan deger, ort ile degistirildi
            counter += 1
    demand["demand_null_sayisi"].values[i] = counter
    counter = 0
    

# demand / tuketim orani -> max orani tut
# tuketimleri 12 aya düsür -> demand / tuketimlerin yillik farki -> 0-1 arası deger alabilir

demand["max_demand/tuketim_orani"] = np.nan

months = []
for x in tuketim1:
    months.append(x)

dmonths = []
for x in demand1:
    dmonths.append(x)
    
len_m = 12 # 12 ay verir
max_percent = 0


for i in range(len(tuketim)): # satir bazinda dolasir
    for j in range(len_m):
        month = months[j]
        nmonth = months[j+12]
        dmonth = dmonths[j]
        
        diff = abs(tuketim[month].values[i] - tuketim[nmonth].values[i])
 
        if(diff >= demand[dmonth].values[i]):
            percent = demand[dmonth].values[i] / diff 
            if(percent > max_percent):
                max_percent = percent
    demand["max_demand/tuketim_orani"].values[i] = max_percent      
    max_percent = 0
    

# kullanıcıların davranışına göre olağandışı (outlier) tüketim yaptıkları ay sayısı hesaplanır

transpozed_df = df.iloc[:, 0:24].T
Q1 = transpozed_df.quantile(0.25)
Q3 = transpozed_df.quantile(0.75)
IQR = Q3 - Q1
demand["OUTLIER_SAYISI"] = ((transpozed_df < (Q1 - 1.5 * IQR)) | (transpozed_df > (Q3 + 1.5 * IQR))).sum()

In [ ]:
# ordinal encoding

df["MAHALLE_RISK_SKORU_group_encoded"] = df["MAHALLE_RISK_SKORU_group"].map(
    {'Düşük':1, 'Orta':2, 'Yüksek':3})

df["SOKAK_RISK_SKORU_group_encoded"] = df["SOKAK_RISK_SKORU_group"].map(
    {'Düşük':1,'Yüksek':2})

df["SOB_RISK_SKORU_group_encoded"] = df["SOB_RISK_SKORU_group"].map(
    {'Çok Düşük':1, 'Düşük':2,'Orta':3, 'Yüksek':4})

df["SAYAC_MARKA_encoded"] = df["SAYAC_MARKA"].map(
    {'Orta - Düşük Risk':1, 'Yüksek Risk':2,'Çok Yüksek Risk':3})

ordinal_columns = ["MAHALLE_RISK_SKORU_group", "SOKAK_RISK_SKORU_group", "SOB_RISK_SKORU_group", "SAYAC_MARKA"]
df = df.drop(ordinal_columns, axis=1)

others = df.iloc[:,36:54].copy() # kalan feature'lari incelemek icin

df = pd.concat([tuketim,demand,others], axis=1)

In [ ]:
# kullanılmayan kolonların drop edilmesi

drops = ["SAYAC_MODEL", "SAYAC_MALZEME_ID",
         "SAYAC_BASLANGIC_TARIHI", "SAYAC_BITIS_TARIHI", 
         "SAYAC_TAKILMA_TARIHI", "SOB_RISK_SKORU", "SOKAK_RISK_SKORU", "MAHALLE_RISK_SKORU"]
df.drop(drops,axis=1,inplace=True)

In [ ]:
# dağılımdaki çarpıklığı azaltma

for i in df.iloc[:, 0:49]:
    df[i] = df[i]**(.5)
    
#çarpıklığı düzeltme işleminden sonra ortaya çıkan 2 null verinin doldurulması
df["TUKETIM_M24"].fillna(18,inplace=True)
df["min_tuketim"].fillna(18,inplace=True)

# OHE yapılması

In [ ]:
cat_cols = ["TESISAT_TIPI","SAYAC_OLCUM_TURU", "SAYAC_FAZ_N", "SAYAC_YAPIM_YILI"]
for col in cat_cols:
    df[col] = df[col].astype('object')

columns_to_encode = []
for col in df.columns:
    if df[col].dtype == 'object':
        columns_to_encode.append(col)
columns_to_encode

In [ ]:
# Kolon isimlerinin sırasını almak için get_dummies yapıyorum.
# OneHotEncoding yapınca kolon isimleri kayboluyor.
df_temp = pd.get_dummies(df, columns = columns_to_encode)
cl=[]
for _ in df_temp.columns:
    cl.append(_)
cl # OHE yapılmış datamızın tüm kolon isimleri.

In [ ]:
num_val = df_temp.iloc[:,:54] # Nümerik verilerimiz tamamı
cat_val = cl[54:] # OHE yapılmış kolonlarımızın isimleri

from sklearn.preprocessing import OneHotEncoder
# Validation kısmında hata almamak için "handle_unknown=ignore" olarak ayarladım.
ohe = OneHotEncoder(handle_unknown='ignore') 
dfohe=ohe.fit_transform(df[columns_to_encode]).toarray() # Object verilerimize OHE uyguladık.
dfohe

In [ ]:
num_val.reset_index(drop=True, inplace=True)
df4=pd.DataFrame(dfohe, columns=cat_val) # OHE yapılmış verilerimizin DF'i
df_test = pd.concat([num_val,df4], axis=1) # Önceden ayırdığımız nümeric veriler ile OHE verimizi birleştirdik.
df_test.head(10) # TRAIN datamızın tüm işlemlerden geçmiş hali bu.

# Veri setinin predictor (X) - target (y) olarak ayrılması

In [ ]:
# Train datamızın bölünmesi
features = []
for col in df_test.columns:
    if col != 'NK_FLAG':
        features.append(col)

target = 'NK_FLAG'

X = df_test[features]
y = df_test[target]

# verilerin scale edilmesi

In [ ]:
# X verimizi scale ediyoruz.
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)

# Tahminleme işlemi ve değerlendirme

In [ ]:
# trainde kaydedilen model dosyası yüklenir

import pickle
model = pickle.load(open(r"C:\Users\Asus\Desktop\Enerjisa_Bootcamp_Bitirme_Projesi_Grup1\logreg_model.pkl","rb"))

In [ ]:
preds = model.predict(X)
score = model.score(X, y)
print(score*100)
print()
print(classification_report(y, preds))

cm = confusion_matrix(y, preds)
sns
sns.heatmap(cm, annot=True, fmt='g')

In [ ]:
threshold = 0.85
predictions_lr = (model.predict_proba(X)[:, 1] >= threshold).astype(int)

cm = confusion_matrix(y, preds)
sns
sns.heatmap(cm, annot=True, fmt='g')

print('Valuation for test data only:')
print(classification_report(y, preds))
print("----------------------------------------------------------------------")
print('Valuation for test data only  (new_threshold):')
print(classification_report(y, predictions_lr))